# Latent variable models and variational autoencoders

We wish to learn latent representations of linguistic data in order to determine whether different lexical strata within I-A languages possess a phonotactic signature that can be detected by deep learning (more or less).

Given our observed data $\boldsymbol x$, we can envision the following generative process for the data:

$$
\boldsymbol z_i \sim P(z_i); \boldsymbol x_i \sim P(x_i|z_i) : i \in \{1,...,N\}
$$

That is to say, for each data point $\boldsymbol x_i$, there is a latent variable $\boldsymbol z_i$ that is independently and identically distributed that can be used to generate $\boldsymbol x_i$. Learning the latent code $\boldsymbol z_i$ for each data point tells us something interesting about patterns found in the data. As a bonus, learning the parameters associated with $P(x|z)$ allows us to generate well-formed instances of $\boldsymbol x$. This model specification also allows us to put a prior over $\boldsymbol z_i$, giving the model a Bayesian flavor. This opens up the possibility of using Variational Inference (VI) to learn model parameters. Under standard gradient-based optimization, we want to find parameters that minimize the loss or maximize the log-likelihood of the data:

$$
\argmax_{\theta} \log P(\text{data}|\theta)
$$

VI on the other hand does not simply compute a point estimate for $\theta$ --- rather, it finds a "variational posterior distribution." This means that we have to decide a priori on the functional form of the posterior distribution (sampling-based Bayesian approaches like MCMC do not require such an assumption). The Normal/Gaussian distribution is a popular choice, as underlyingly Gaussian variates can be transformed to have different types of support (e.g., with the Softmax function to represent probability vectors, etc.). This means that we are not optimizing $\theta$ but a mean $\mu_\theta$ and st.dev. $\sigma_\theta$, if we assume a Gaussian variational posterior. Stochastic Variational Inference tries to optimize the following:

$$
\argmax_{\mu_\theta,\sigma_\theta} \log \mathbb{E}_{z \sim \mathcal{N}(\mu_\theta,\sigma_\theta)} P(\text{data}|z) - D_{KL}(\mathcal{N}(\mu_\theta,\sigma_\theta)||\mathcal{N}(0,1))
$$

Here, the likelihood term is $\mathbb{E}_{z \sim \mathcal{N}(\mu_\theta,\sigma_\theta)} P(\text{data}|z)$, which represents the expected value of the likelihood under samples from the variational distribution $\mathcal{N}(\mu_\theta,\sigma_\theta)$ (in different circumstances you need different numbers of Monte Carlo samples to compute the expected value). The second term is the Kullback-Leibler divergence between the variational posterior and the prior (here assumed to be standard normal). This term is the "evidence lower bound" (ELBO).

In theory, we could use an inference algorithm like Stochastic Variational Inference (SVI) to optimize the parameters directly via the ELBO. This would give us a pair of parameters $\mu_\theta^{(i)}$, $\sigma_\theta^{(i)}$ for each data point. But this is computationally quite challenging.

In order to alleviate this problem, auto-encoding variational bayes or amortized variational inference comes to the rescue. Like its autoencoder namesake, AEVB learns $\mu_\theta,\sigma_\theta$ *from the data* through an "encoder network", which can then be used to generate samples of $\theta$, which can then be used to reconstruct the data via the "decoder network" $P(x|\theta)$. Both the encoder and decoder networks are some sort of neural network (an MLP in the most basic type of model).

This gives us the variational autoencoder (VAE), which has the following generative process for each data point (I drop indices for readability):

* $\mu_\theta,\sigma_\theta := \text{Encoder}(x)$
* $\theta \sim \mathcal{N}(\mu_\theta,\sigma_\theta)$ (or under the re-parameterization trick, which allows all differentiable variables to be deterministic, $\theta := \mu_\theta + \sigma_\theta \epsilon, \epsilon \sim \mathcal{N}(0,1)$
* $\hat{x} := \text{Decoder}(\theta)$

All we have to do is find parameters for the Encoder and Decoder networks that minimize the loss between $\hat{x}$ and $x$ under samples from the variational distribution.

Thus, you have the Gaussian VAE, which is easy to set up and for which there are ample tutorials. This model has been used in some NLP applications, with varying success. Regarding the question of how a word's phonotactics situate it in latent space, it is not clear to me how theoretically interesting the Gaussian VAE is (in comparison to a standard AE), unless there is some clear operationalization of the "Bayesian"/uncertainty-based part of the model --- e.g., it could be that certain words have an ambiguous phonotactic profile, which is reflected by larger values of $\sigma_\theta$.

However, the VAE architecture presents a number of attractive features. One of these is the ability to place a prior over the latent variable, as long as the distribution has a differentiable non-centered parameterization.

## Latent discrete variables

Latent continuous variables are flexible and easy to learn, but sometimes difficult to determine. Particularly when investigating lexical strata, a discrete latent variable is preferable, where a form is associated with one of $K$ groups. The generative process is something like 

$$
\boldsymbol z_i \sim \text{Categorical}(\theta); \boldsymbol x_i \sim P(x_i|z_i) : i \in \{1,...,N\}
$$

Discrete latent variables pose problems for gradient-based inference algorithms, given their non-differentiability. There are some work-arounds. In theory the standard REINFORCE gradient can be used, but this exhibits bias and high variance. Other tricks are possible, such as straight-through estimation, which computes the loss on the basis of a discretized representation of a latent variable but updates gradients on the basis of a continuous representation. However, the straight-through estimator is biased as well. Additional tricks (e.g., Gumbel-Softmax, RELAX, REBAR) have come up with various ways to reduce variance in this type of estimator.

An approach that does not suffer from these problems is to marginalize out the discrete variable by summing, yielding the following likelihood:

$$
P(x_i|\theta) = \sum_{k=1}^{K} P(x_i,z=k|\theta) = \sum_{k=1}^{K} P(x_i|z=k)P(z=k|\theta)
$$


PyTorch and Keras make it really difficult to optimize custom loss functions that cannot be defined simply via a forward pass. Pyro allows you to define models with categorical latent variables, and it takes care of the marginalization for you. Tensorflow (<2.0) makes custom loss functions of this type easy to use, but I never figured out how to do this without having to define neural architectures from scratch.
